# Model with Metrics

## Dependencies

```pip install seldon-core```

## Summary of Custom Metrics

Example testing a model with custom metrics.

Metrics can be 

  * A ```COUNTER``` : the returned value will increment the current value
  * A ```GAUGE``` : the returned value will overwrite the current value
  * A ```TIMER``` : a number of millisecs. Prometheus SUM and COUNT metrics will be created.
  
You need to provide a list of dictionaries each with the following:

  * a ```type``` : COUNTER, GAUGE, or TIMER
  * a ```key``` : a user defined key
  * a ```value``` : a float value
  
See example code below:
 

In [1]:
!pygmentize ModelWithMetrics.py

class ModelWithMetrics(object):

    def __init__(self):
        print("Initialising")

    def predict(self,X,features_names):
        print("Predict called")
        return X

    def metrics(self):
        return [
            {"type":"COUNTER","key":"mycounter","value":1}, # a counter which will increase by the given value
            {"type":"GAUGE","key":"mygauge","value":100}, # a gauge which will be set to given value
            {"type":"TIMER","key":"mytimer","value":20.2}, # a timer which will add sum and count metrics - assumed millisecs
            ]
    


## REST

In [2]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.6 model-with-metrics-rest:0.1

---> Installing application source...
Build completed successfully


In [3]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-rest:0.1

d3b8b00d7c03c219b6bd69e4eaaaa61dfb57ae550d4da5db1b050b4a72082519


### Test predict

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[4.744 4.857 1.579 0.458]]
RECEIVED RESPONSE:
meta {
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
    value: 20.200000762939453
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  ndarray {
    values {
      list_value {
        values {
          number_value: 4.744
        }
        values {
          number_value: 4.857
        }
        values {
          number_value: 1.579
        }
        values {
          number_value: 0.458
        }
      }
    }
  }
}




In [5]:
!docker rm model-with-metrics --force

model-with-metrics


## gRPC

In [6]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.6 model-with-metrics-grpc:0.1

---> Installing application source...
Build completed successfully


In [7]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-grpc:0.1

85ac344a48c56931ff3bef98ca02bb0d9f7e7198619fee933faf92d62c594080


### Test predict

In [8]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[7.607 2.626 3.027 2.034]]
RECEIVED RESPONSE:
meta {
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
    value: 20.200000762939453
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  ndarray {
    values {
      list_value {
        values {
          number_value: 7.607
        }
        values {
          number_value: 2.626
        }
        values {
          number_value: 3.027
        }
        values {
          number_value: 2.034
        }
      }
    }
  }
}




In [9]:
!docker rm model-with-metrics --force

model-with-metrics


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [10]:
!minikube start --memory 4096

There is a newer version of minikube available (v0.35.0).  Download it here:
https://github.com/kubernetes/minikube/releases/tag/v0.35.0

To disable this notification, run the following:
minikube config set WantUpdateNotification false
😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [11]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [12]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [13]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [14]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Thu Mar 14 10:57:08 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5554c4d8b6-stv22  0/1    ContainerCreating  0         0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          0s


NOTES:
NOTES: TODO


In [15]:
!helm install seldon-core-analytics --name seldon-core-analytics --set grafana_prom_admin_password=password --set persistence.enabled=false --repo https://storage.googleapis.com/seldon-charts 

NAME:   seldon-core-analytics
LAST DEPLOYED: Thu Mar 14 10:57:17 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           1s

==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.109.210.152  <none>       80/TCP        1s
grafana-prom              NodePort   10.109.150.113  <none>       80:32474/TCP  0s
prometheus-node-exporter  ClusterIP  None            <none>       9100/TCP      0s
prometheus-seldon         ClusterIP  10.111.102.97   <none>       80/TCP        0s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-666rl      0/1    ContainerCreating  0         1s
alertmanager-deployment-b9c99d4-p5cfz     0/1    ContainerCreating  0         1s
grafana-prom-deployment-747445685c-45sdz  0/1    ContainerCre

 * Port forward the dashboard when running
  ```
     kubectl port-forward $(kubectl get pods -n default -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n default 3000:3000
  ```
  * Visit http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1 and login using "admin" and the password you set above when launching with helm.

## REST

In [17]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.6 model-with-metrics-rest:0.1

---> Installing application source...
Build completed successfully


In [18]:
!kubectl create -f deployment-rest.json

seldondeployment.machinelearning.seldon.io/mymodel created


In [19]:
!kubectl rollout status deploy/mymodel-mymodel-b79af31

deployment "mymodel-mymodel-b79af31" successfully rolled out


### Test predict

In [20]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:

[[5.659 2.404 1.168 0.075]]
RECEIVED RESPONSE:
meta {
  puid: "riri721bkhsum9fopcdie13hm2"
  requestPath {
    key: "complex-model"
    value: "model-with-metrics-rest:0.1"
  }
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
    value: 20.200000762939453
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  tensor {
    shape: 1
    shape: 4
    values: 5.659
    values: 2.404
    values: 1.168
    values: 0.075
  }
}




In [21]:
!kubectl delete -f deployment-rest.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


## gRPC

In [22]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.6 model-with-metrics-grpc:0.1

---> Installing application source...
Build completed successfully


In [23]:
!kubectl create -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io/mymodel created


In [24]:
!kubectl rollout status deploy/mymodel-mymodel-5818788

deployment "mymodel-mymodel-5818788" successfully rolled out


### Validate on Grafana

To check the metrics have appeared on Prometheus and are available in Grafana you could create a new graph in a dashboard and use the query:

```
mycounter_total
```


### Test predict

In [25]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

----------------------------------------
SENDING NEW REQUEST:

[[4.97  3.561 6.3   0.898]]
RECEIVED RESPONSE:
meta {
  puid: "9toeq43ol6l6kj4ut2q4ded2bo"
  requestPath {
    key: "complex-model"
    value: "model-with-metrics-grpc:0.1"
  }
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
    value: 20.200000762939453
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  tensor {
    shape: 1
    shape: 4
    values: 4.97
    values: 3.561
    values: 6.3
    values: 0.898
  }
}




In [26]:
!kubectl delete -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


In [27]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
